In [1]:
import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import cross_validate, train_test_split
import datetime
from sklearn.neural_network import MLPRegressor
import lightgbm as lgb
import time
import pytz
from sklearn.preprocessing import StandardScaler



In [2]:
link_gohawks = "./ECE219_tweet_data/tweets_#gohawks.txt"
link_gopatriots = "./ECE219_tweet_data/tweets_#gopatriots.txt"
link_nfl = "./ECE219_tweet_data/tweets_#nfl.txt"
link_patriots = "./ECE219_tweet_data/tweets_#patriots.txt"
link_sb49 = "./ECE219_tweet_data/tweets_#sb49.txt"
link_superbowl = "./ECE219_tweet_data/tweets_#superbowl.txt"

In [3]:
#@markdown Helper functions
#@markdown - Read file and return `json_file`.
def read_file(link):
  json_file = []
  with open(link) as f:
    for line in f:
      json_file.append(json.loads(line))
  return json_file


In [13]:
tweet_sb49 = read_file(link_sb49)


In [18]:
tweet_sb49[100]["tweet"].keys()

dict_keys(['contributors', 'truncated', 'text', 'in_reply_to_status_id', 'id', 'favorite_count', 'source', 'retweeted', 'coordinates', 'timestamp_ms', 'entities', 'in_reply_to_screen_name', 'in_reply_to_user_id', 'retweet_count', 'id_str', 'favorited', 'user', 'geo', 'in_reply_to_user_id_str', 'possibly_sensitive', 'lang', 'created_at', 'filter_level', 'in_reply_to_status_id_str', 'place'])

In [19]:
tweet_sb49[53400]['tweet']['entities']

{'symbols': [],
 'user_mentions': [{'indices': [13, 22],
   'id_str': '31126587',
   'screen_name': 'Patriots',
   'name': 'New England Patriots',
   'id': 31126587},
  {'indices': [51, 55],
   'id_str': '19426551',
   'screen_name': 'nfl',
   'name': 'NFL',
   'id': 19426551}],
 'trends': [],
 'hashtags': [{'indices': [31, 36], 'text': 'SB49'},
  {'indices': [38, 50], 'text': 'PatriotsWIN'}],
 'urls': [{'indices': [56, 78],
   'url': 'http://t.co/J2DGcgokGN',
   'expanded_url': 'http://bit.ly/1CfcIQw',
   'display_url': 'bit.ly/1CfcIQw'}]}

In [20]:
tweet_sb49[0].keys()

dict_keys(['firstpost_date', 'title', 'url', 'tweet', 'author', 'original_author', 'citation_date', 'metrics', 'highlight', 'type', 'citation_url'])

In [22]:
tweets = []
impressions = []
usermentions = []
hashtags = []
posttime = []
for i in tweet_sb49:
    tweets.append(i["tweet"]["text"])
    impressions.append(i["metrics"]["impressions"])
    um = i['tweet']['entities']['user_mentions']
    temp_um = []
    for j in um:
        temp_um.append(j["screen_name"])
    usermentions.append(temp_um)
    ht = i['tweet']['entities']['hashtags']
    temp_ht = []
    for j in ht:
        temp_ht.append(j['text'])
    hashtags.append(temp_ht)
    posttime.append(i["citation_date"])

In [23]:
temp_time = []
temp_time.append(time.mktime(datetime.datetime(2015, 2, 2, 0, 12, 35, 0).timetuple()))
temp_time.append(time.mktime(datetime.datetime(2015, 2, 2, 0, 32, 20, 0).timetuple()))
temp_time.append(time.mktime(datetime.datetime(2015, 2, 2, 0, 53, 27, 0).timetuple()))
temp_time.append(time.mktime(datetime.datetime(2015, 2, 2, 0, 58, 28, 0).timetuple()))
temp_time.append(time.mktime(datetime.datetime(2015, 2, 2, 1, 41, 7, 0).timetuple()))
temp_time.append(time.mktime(datetime.datetime(2015, 2, 2, 1, 58, 59, 0).timetuple()))
temp_time.append(time.mktime(datetime.datetime(2015, 2, 2, 2, 27, 47, 0).timetuple()))
temp_time.append(time.mktime(datetime.datetime(2015, 2, 2, 2, 47, 25, 0).timetuple()))

score_time = {
    temp_time[0]: (7, 0),
    temp_time[1]: (7, 7),
    temp_time[2]: (14, 7),
    temp_time[3]: (14, 14),
    temp_time[4]: (14, 17),
    temp_time[5]: (14, 24),
    temp_time[6]: (21, 24),
    temp_time[7]: (28, 24)
}
scorechange_time = list(score_time.keys())


score = []

for i in range(0,len(posttime)):
    if posttime[i] < scorechange_time[0]:
        score.append((0,0))       
    if posttime[i] >= scorechange_time[0] and posttime[i] < scorechange_time[1]:
        score.append(score_time[scorechange_time[0]])
    if posttime[i] >= scorechange_time[1] and posttime[i] < scorechange_time[2]:
        score.append(score_time[scorechange_time[1]])
    if posttime[i] >= scorechange_time[2] and posttime[i] < scorechange_time[3]:
        score.append(score_time[scorechange_time[2]])
    if posttime[i] >= scorechange_time[3] and posttime[i] < scorechange_time[4]:
        score.append(score_time[scorechange_time[3]])
    if posttime[i] >= scorechange_time[4] and posttime[i] < scorechange_time[5]:
        score.append(score_time[scorechange_time[4]])
    if posttime[i] >= scorechange_time[5] and posttime[i] < scorechange_time[6]:
        score.append(score_time[scorechange_time[5]])
    if posttime[i] >= scorechange_time[6] and posttime[i] < scorechange_time[7]:
        score.append(score_time[scorechange_time[6]])
    if posttime[i] >= scorechange_time[7]:
        score.append(score_time[scorechange_time[7]])
    
    

In [24]:
df = pd.DataFrame(list(zip(tweets, impressions, usermentions, hashtags, posttime)),
               columns =['tweets', 'impressions', 'usermentions', 'hashtags', 'time'])

In [25]:
patriots = ["patriots", "patriotsnation", "gopatriots" "tombrady", "newenglandpatriots", "goat", "foreverne","gostkowski", "blafell1", "robgronkowski", "dannyamendola", "edelman11", "patriotsbeat"]
patriots_str = " ".join(patriots)
hawks = ["seahawks", "gohawks", "seattle", "eagles", "wilson", "moneylynch", "stephenHauschka", "hardballchris", "dangerusswilson", "dougbaldwinjr"]
hawks_str = " ".join(hawks)

In [26]:
def pcount(lst):
    count = 0
    for i in lst:
        if i.lower() in patriots_str:
            count += 1
    return count
def hcount(lst):
    count = 0
    for i in lst:
        if i.lower() in hawks_str:
            count += 1
    return count

In [27]:
df['npatriots_ht'] = df['hashtags'].apply(lambda x: pcount(x))
df['nhawks_ht'] = df['hashtags'].apply(lambda x: hcount(x))
df['npatriots_m'] = df['usermentions'].apply(lambda x: pcount(x))
df['nhawks_m'] = df['usermentions'].apply(lambda x: hcount(x))
df["score"] = score

In [28]:
df["t"] = df["time"].apply(lambda x: str(pd.Timestamp(x, unit='s'))[:-2] + "00")

In [29]:
tag_search = list(set([a for b in df.hashtags.tolist() for a in b]))

import re
hawk_tags = []
patriot_tags = []
for t in tag_search:
    if re.search('hawk', t, re.IGNORECASE):
        hawk_tags.append(t)
    elif re.search('patriot', t, re.IGNORECASE):
        patriot_tags.append(t)

hawk_tags = list(set(hawk_tags + hawks))
patriot_tags = list(set(patriot_tags + patriots))
len(hawk_tags), len(patriot_tags)

(819, 473)

In [30]:
def classify_team(lst):
    s1 = set(lst)
    s2 = set(hawk_tags)
    s3 = set(patriot_tags)
    # does not tag a specific team or tag an equal number of times -> both
    if not s1 & s2 and not s1 & s3:
        return 'both'
    if len(s1 & s2) > len(s1 & s3):
        return 'hawks'
    elif len(s1 & s2) < len(s1 & s3):
        return 'patriot'
    else:
        return 'both'

df['team'] = df['hashtags'].apply(classify_team)
df.team.value_counts()

both       329633
hawks      230670
patriot    183346
Name: team, dtype: int64

In [31]:
# from nltk.corpus import stopwords
def clean_tweet(tweet):
    if type(tweet) == np.float:
        return ""
    temp = tweet.lower()
    temp = re.sub("'", "", temp) # to avoid removing contractions in english
    temp = re.sub("@[A-Za-z0-9_]+","", temp)
    temp = re.sub("#[A-Za-z0-9_]+","", temp)
    temp = re.sub(r'http\S+', '', temp)
    temp = re.sub('[()!?]', ' ', temp)
    temp = re.sub('\[.*?\]',' ', temp)
    temp = re.sub("[^a-z0-9]"," ", temp)
    temp = temp.split()
    # temp = [w for w in temp if not w in stopwords]
    temp = " ".join(word for word in temp)
    return temp

# https://catriscode.com/2021/05/01/tweets-cleaning-with-python/

In [32]:
df['cleaned_tweets'] = df.tweets.apply(clean_tweet)
df.cleaned_tweets[0]

/var/folders/t4/781mdk1946v_3wdj1lyyf5_m0000gn/T/ipykernel_7026/1771556743.py:3: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if type(tweet) == np.float:


'id take a day off on feb 2 if the and or the advance to the'

In [33]:
# use textblob
# https://towardsdatascience.com/the-most-favorable-pre-trained-sentiment-classifiers-in-python-9107c06442c6
from textblob import TextBlob
df['sentiment'] = df['tweets'].apply(lambda text: TextBlob(text).sentiment.polarity)

In [34]:
df["score_p"] = df['score'].apply(lambda x: x[0])
df["score_h"] = df['score'].apply(lambda x: x[1])

In [35]:
df

,tweets,impressions,usermentions,hashtags,time,npatriots_ht,nhawks_ht,npatriots_m,nhawks_m,score,t,team,cleaned_tweets,sentiment,score_p,score_h
0,I'd take a day off on Feb 2 if the #Patriots a...,69,[],"[Patriots, Seahawks, SuperBowl, NFL, SB49]",1421238675,1,1,0,0,"(0, 0)",2015-01-14 12:31:00,both,id take a day off on feb 2 if the and or the a...,0.000000,0,0
1,Looking for blog post ideas for the Superbowl?...,21,"[AZSuperBowl, SuperBowl]","[SB49, AZSuperBowl, blogpostideas]",1421244058,0,0,0,0,"(0, 0)",2015-01-14 14:00:00,both,looking for blog post ideas for the superbowl,0.000000,0,0
2,I wore my @PatMcAfeeShow shirt and my new Colt...,53,[PatMcAfeeShow],"[GoColts, BoomStick, SB49, Lombardi]",1421246898,0,0,0,0,"(0, 0)",2015-01-14 14:48:00,both,i wore my shirt and my new colts warmup last s...,0.045455,0,0
3,"""@Duracell: Play #Madden15 vs real NFL players...",545,"[Duracell, GregxRuiz]","[Madden15, SB49]",1421249217,0,0,0,0,"(0, 0)",2015-01-14 15:26:00,both,play vs real nfl players for a chance to win t...,0.500000,0,0
4,“@Duracell: Play #Madden15 vs real NFL players...,254,"[Duracell, CMStud]","[Madden15, SB49, Duracell26hrs]",1421249288,0,0,0,0,"(0, 0)",2015-01-14 15:28:00,both,play vs real nfl players for a chance to win t...,0.500000,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
743644,Media coverage by @12News of Last week's #SB49...,154,"[12News, JGroupMedia]","[SB49, jenublasi, trunkclub, Fanfest]",1423335031,0,0,0,0,"(28, 24)",2015-02-07 18:50:00,both,media coverage by of last weeks s host styled ...,0.000000,28,24
743645,@PatriotsBeat Incredible fan's eye view of #SB...,495,[PatriotsBeat],"[SB49, Patriots, Seahawks, PatriotsNation]",1423335300,2,1,1,0,"(28, 24)",2015-02-07 18:55:00,patriot,incredible fans eye view of beat the everyone ...,0.900000,28,24
743646,Huge touchdown by our guy @dannyamendola Don’t...,34,[DannyAmendola],"[SB49, NFLMobile]",1423335301,0,0,1,0,"(28, 24)",2015-02-07 18:55:00,both,huge touchdown by our guy don t miss him make ...,0.200000,28,24
743647,That’s 7 for the @Seahawks! Does this change y...,34,[Seahawks],"[WhosGonnaWin, SB49]",1423335329,0,0,0,1,"(28, 24)",2015-02-07 18:55:00,both,that s 7 for the does this change your pick,0.000000,28,24


In [36]:
# data = pd.DataFrame(df.groupby(['team', 't'])['sentiment'].mean())
# data['count'] = pd.DataFrame(df.groupby(['team', 't'])['tweets'].count())
# data['timestamp'] = pd.DataFrame(df.groupby(['team', 't'])['time'].max())
# data["npatriots_ht"] = pd.DataFrame(df.groupby(['team', 't'])['npatriots_ht'].sum())
# data["nhawks_ht"] = pd.DataFrame(df.groupby(['team', 't'])['nhawks_ht'].sum())
# data["npatriots_m"] = pd.DataFrame(df.groupby(['team', 't'])['npatriots_m'].sum())
# data["nhawks_m"] = pd.DataFrame(df.groupby(['team', 't'])['nhawks_m'].sum())
# data["score_diff"] = pd.DataFrame(df.groupby(['team', 't'])['score'].max())
# data["score_p"] = pd.DataFrame(df.groupby(['team', 't'])['score_p'].max())
# data["score_h"] = pd.DataFrame(df.groupby(['team', 't'])['score_h'].max())
# data["score_diff"] = data.score_p - data.score_h
# data = data.reset_index(level=[1])
# data

data = pd.DataFrame(df.groupby(['time'])['sentiment'].mean())
data['count'] = pd.DataFrame(df.groupby([ 'time'])['tweets'].count())
data['timestamp'] = pd.DataFrame(df.groupby([ 'time'])['time'].max())
data["npatriots_ht"] = pd.DataFrame(df.groupby(['time'])['npatriots_ht'].sum())
data["nhawks_ht"] = pd.DataFrame(df.groupby(['time'])['nhawks_ht'].sum())
data["npatriots_m"] = pd.DataFrame(df.groupby(['time'])['npatriots_m'].sum())
data["nhawks_m"] = pd.DataFrame(df.groupby(['time'])['nhawks_m'].sum())
data["score_diff"] = pd.DataFrame(df.groupby(['time'])['score'].max())
data["score_p"] = pd.DataFrame(df.groupby([ 'time'])['score_p'].max())
data["score_h"] = pd.DataFrame(df.groupby(['time'])['score_h'].max())
data["score_diff"] = data.score_p - data.score_h
data = data.reset_index()
data

,time,sentiment,count,timestamp,npatriots_ht,nhawks_ht,npatriots_m,nhawks_m,score_diff,score_p,score_h
0,1421238675,0.000000,1,1421238675,1,1,0,0,0,0,0
1,1421244058,0.000000,1,1421244058,0,0,0,0,0,0,0
2,1421246898,0.045455,1,1421246898,0,0,0,0,0,0,0
3,1421249217,0.500000,1,1421249217,0,0,0,0,0,0,0
4,1421249288,0.500000,1,1421249288,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
117774,1423335031,0.000000,1,1423335031,0,0,0,0,4,28,24
117775,1423335300,0.900000,1,1423335300,2,1,1,0,4,28,24
117776,1423335301,0.200000,1,1423335301,0,0,1,0,4,28,24
117777,1423335329,0.000000,1,1423335329,0,0,0,1,4,28,24


In [37]:
data

,time,sentiment,count,timestamp,npatriots_ht,nhawks_ht,npatriots_m,nhawks_m,score_diff,score_p,score_h
0,1421238675,0.000000,1,1421238675,1,1,0,0,0,0,0
1,1421244058,0.000000,1,1421244058,0,0,0,0,0,0,0
2,1421246898,0.045455,1,1421246898,0,0,0,0,0,0,0
3,1421249217,0.500000,1,1421249217,0,0,0,0,0,0,0
4,1421249288,0.500000,1,1421249288,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
117774,1423335031,0.000000,1,1423335031,0,0,0,0,4,28,24
117775,1423335300,0.900000,1,1423335300,2,1,1,0,4,28,24
117776,1423335301,0.200000,1,1423335301,0,0,1,0,4,28,24
117777,1423335329,0.000000,1,1423335329,0,0,0,1,4,28,24


In [38]:
# Standardize
scale= StandardScaler()
# select competition time
start_time = int(time.mktime(datetime.datetime(2015, 2, 1, 23, 30, 00, 00).timetuple()))
end_time = int(time.mktime(datetime.datetime(2015, 2, 2, 2, 50, 25, 0).timetuple()))

data = data[data["timestamp"] >= start_time]
data = data[data["timestamp"] <= end_time]

data

,time,sentiment,count,timestamp,npatriots_ht,nhawks_ht,npatriots_m,nhawks_m,score_diff,score_p,score_h
82015,1422862200,0.000000,1,1422862200,0,0,0,0,0,0,0
82016,1422862201,0.600000,1,1422862201,0,0,0,0,0,0,0
82017,1422862208,0.066667,3,1422862208,0,1,0,0,0,0,0
82018,1422862209,0.000000,1,1422862209,0,0,0,0,0,0,0
82019,1422862211,0.000000,1,1422862211,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
85032,1422874212,0.000000,1,1422874212,1,0,0,0,4,28,24
85033,1422874217,0.000000,1,1422874217,0,0,0,0,4,28,24
85034,1422874218,0.000000,1,1422874218,0,0,0,0,4,28,24
85035,1422874219,0.000000,1,1422874219,0,0,0,0,4,28,24


### LGBM

In [40]:
lgb_data = data
def lag_features(dataframe, lags):
    for lag in lags:
        dataframe['scores_lag_p_' + str(lag)] = dataframe['score_p'].shift(lag)
        dataframe['scores_lag_h_' + str(lag)] = dataframe['score_h'].shift(lag)
    return dataframe
lgb_data = lag_features(lgb_data, [1,2,3,4,5,6,7,8,9,10])

In [41]:
def roll_mean_features(dataframe, windows):
    for window in windows:
        dataframe['scores_roll_mean_p' + str(window)] = dataframe['score_p'].shift(1).rolling(window=window, win_type="triang").mean()
        dataframe['scores_roll_mean_h' + str(window)] = dataframe['score_h'].shift(1).rolling(window=window, win_type="triang").mean()
    return dataframe
lgb_data = roll_mean_features(lgb_data, [5, 10])

In [72]:
lgb_data.columns

Index(['time', 'sentiment', 'count', 'timestamp', 'npatriots_ht', 'nhawks_ht',
       'npatriots_m', 'nhawks_m', 'score_diff', 'score_p', 'score_h',
       'scores_lag_p_1', 'scores_lag_h_1', 'scores_lag_p_2', 'scores_lag_h_2',
       'scores_lag_p_3', 'scores_lag_h_3', 'scores_lag_p_4', 'scores_lag_h_4',
       'scores_lag_p_5', 'scores_lag_h_5', 'scores_lag_p_6', 'scores_lag_h_6',
       'scores_lag_p_7', 'scores_lag_h_7', 'scores_lag_p_8', 'scores_lag_h_8',
       'scores_lag_p_9', 'scores_lag_h_9', 'scores_lag_p_10',
       'scores_lag_h_10', 'scores_roll_mean_p5', 'scores_roll_mean_h5',
       'scores_roll_mean_p10', 'scores_roll_mean_h10'],
      dtype='object')

In [54]:
y = lgb_data.score_p
cols = [col for col in lgb_data.columns if col not in ['t', "timestamp", "score_p", "score_h", "score_diff"]]
X = lgb_data[cols]

X_train, X_test, y_train, y_test_p = train_test_split(X, y, test_size=0.2)

lgbtrain = lgb.Dataset(data=X_train, label=y_train, feature_name=cols)

lgbval = lgb.Dataset(data=X_test, label=y_test_p, reference=lgbtrain, feature_name=cols)

lgb_params = {'num_leaves': 10,
              'learning_rate': 0.02,
              'feature_fraction': 0.8,
              'max_depth': 5,
              'verbose': 0,
              'num_boost_round': 10000,
              'early_stopping_rounds': 200,
              'nthread': -1}
def smape(preds, target):
    n = len(preds)
    masked_arr = ~((preds == 0) & (target == 0))
    preds, target = preds[masked_arr], target[masked_arr]
    num = np.abs(preds - target)
    denom = np.abs(preds) + np.abs(target)
    smape_val = (200 * np.sum(num / denom)) / n
    return smape_val


def lgbm_smape(preds, train_data):
    labels = train_data.get_label()
    smape_val = smape(preds, labels)
    return 'SMAPE', smape_val, False

model = lgb.train(lgb_params, lgbtrain,
                  valid_sets=[lgbtrain, lgbval],
                  num_boost_round=lgb_params['num_boost_round'],
                  early_stopping_rounds=lgb_params['early_stopping_rounds'],
                  feval=lgbm_smape,
                  verbose_eval=100)

y_pred_val_p = model.predict(X_test, num_iteration=model.best_iteration)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/lightgbm/engine.py:177: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and wil

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001214 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
Training until validation scores don't improve for 200 rounds
[100]	training's l2: 0.963478	training's SMAPE: 60.4877	valid_1's l2: 0.851449	valid_1's SMAPE: 55.8262
[200]	training's l2: 0.0750068	training's SMAPE: 57.4846	valid_1's l2: 0.018974	valid_1's SMAPE: 52.7697
[300]	training's l2: 0.0568177	training's SMAPE: 57.1142	valid_1's l2: 0.00489332	valid_1's SMAPE: 52.4057
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warnin

In [55]:
rmse = np.sqrt(np.mean((y_pred_val_p-y_test_p)**2))
rmse

0.06800329637167757

In [58]:
y = lgb_data.score_h
cols = [col for col in lgb_data.columns if col not in ['t', "timestamp", "score_p", "score_h", "score_diff"]]
X = lgb_data[cols]

X_train, X_test, y_train, y_test_h = train_test_split(X, y, test_size=0.2)

lgbtrain = lgb.Dataset(data=X_train, label=y_train, feature_name=cols)

lgbval = lgb.Dataset(data=X_test, label=y_test_h, reference=lgbtrain, feature_name=cols)

lgb_params = {'num_leaves': 10,
              'learning_rate': 0.02,
              'feature_fraction': 0.8,
              'max_depth': 5,
              'verbose': 0,
              'num_boost_round': 10000,
              'early_stopping_rounds': 200,
              'nthread': -1}
def smape(preds, target):
    n = len(preds)
    masked_arr = ~((preds == 0) & (target == 0))
    preds, target = preds[masked_arr], target[masked_arr]
    num = np.abs(preds - target)
    denom = np.abs(preds) + np.abs(target)
    smape_val = (200 * np.sum(num / denom)) / n
    return smape_val


def lgbm_smape(preds, train_data):
    labels = train_data.get_label()
    smape_val = smape(preds, labels)
    return 'SMAPE', smape_val, False

model = lgb.train(lgb_params, lgbtrain,
                  valid_sets=[lgbtrain, lgbval],
                  num_boost_round=lgb_params['num_boost_round'],
                  early_stopping_rounds=lgb_params['early_stopping_rounds'],
                  feval=lgbm_smape,
                  verbose_eval=100)

y_pred_val_h = model.predict(X_test, num_iteration=model.best_iteration)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/lightgbm/engine.py:177: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and wil

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001268 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
Training until validation scores don't improve for 200 rounds
[100]	training's l2: 1.62055	training's SMAPE: 83.7525	valid_1's l2: 1.55474	valid_1's SMAPE: 79.0567
[200]	training's l2: 0.0772555	training's SMAPE: 80.7047	valid_1's l2: 0.041182	valid_1's SMAPE: 75.936
[300]	training's l2: 0.0475278	training's SMAPE: 80.3629	valid_1's l2: 0.0146158	valid_1's SMAPE: 75.5896
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] N

In [59]:
rmse = np.sqrt(np.mean((y_pred_val_h-y_test_h)**2))
rmse

0.11860027559609429

### prediction

In [60]:
X = data[['sentiment', 'count', 'npatriots_ht','nhawks_ht','npatriots_m', 'nhawks_m']]
y = data.score_h
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
X_train = scale.fit_transform(X_train)
X_test = scale.fit_transform(X_test)
reg = Ridge().fit(X_train, y_train)
predicted_h = reg.predict(X_test)
rmse = np.sqrt(np.mean((predicted_h-y_test)**2))
rmse

9.664058014118703

In [61]:
X = data[['sentiment', 'count', 'npatriots_ht','nhawks_ht','npatriots_m', 'nhawks_m']]
y = data.score_p
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
X_train = scale.fit_transform(X_train)
X_test = scale.fit_transform(X_test)
reg = Ridge().fit(X_train, y_train)
predicted_p = reg.predict(X_test)
rmse = np.sqrt(np.mean((predicted_p-y_test)**2))
rmse

7.15193127678699

In [62]:
y = data.score_diff
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
pred_score_diff = predicted_p - predicted_h
rmse = np.sqrt(np.mean((pred_score_diff-y_test)**2))
rmse

4.778269373977222

In [65]:
X = data[['sentiment', 'count', 'npatriots_ht','nhawks_ht','npatriots_m', 'nhawks_m']]
y = data.score_h
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
X_train = scale.fit_transform(X_train)
X_test = scale.fit_transform(X_test)
reg = LinearRegression().fit(X_train, y_train)
predicted_h = reg.predict(X_test)
rmse = np.sqrt(np.mean((predicted_h-y_test)**2))
rmse

9.432476520106274

In [66]:
X = data[['sentiment', 'count', 'npatriots_ht','nhawks_ht','npatriots_m', 'nhawks_m']]
y = data.score_p
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
X_train = scale.fit_transform(X_train)
X_test = scale.fit_transform(X_test)
reg = LinearRegression().fit(X_train, y_train)
predicted_p = reg.predict(X_test)
rmse = np.sqrt(np.mean((predicted_p-y_test)**2))
rmse

7.189720904635113

In [67]:
y = data.score_diff
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
pred_score_diff = predicted_p - predicted_h
rmse = np.sqrt(np.mean((pred_score_diff-y_test)**2))
rmse

4.366359378410997

In [68]:
X = data[['sentiment', 'count', 'npatriots_ht','nhawks_ht','npatriots_m', 'nhawks_m']]
y = data.score_h
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
X_train = scale.fit_transform(X_train)
X_test = scale.fit_transform(X_test)
reg = Lasso().fit(X_train, y_train)
predicted_h = reg.predict(X_test)
rmse = np.sqrt(np.mean((predicted_h-y_test)**2))
rmse

9.44984650293205

In [69]:
X = data[['sentiment', 'count', 'npatriots_ht','nhawks_ht','npatriots_m', 'nhawks_m']]
y = data.score_p
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
X_train = scale.fit_transform(X_train)
X_test = scale.fit_transform(X_test)
reg = Lasso().fit(X_train, y_train)
predicted_p = reg.predict(X_test)
rmse = np.sqrt(np.mean((predicted_p-y_test)**2))
rmse

7.116806336057419

In [70]:
y = data.score_diff
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
pred_score_diff = predicted_p - predicted_h
rmse = np.sqrt(np.mean((pred_score_diff-y_test)**2))
rmse

4.6040034038163204